In [ ]:
#Imports
!pip install stop_words
import requests
import numpy as np
import pandas as pd
import json
import glob

from google.colab import drive

import re
import nltk
from nltk.stem import WordNetLemmatizer 
from stop_words import get_stop_words
import string
from string import digits
from io import StringIO
from html.parser import HTMLParser
import tensorflow as tf
from tensorflow import keras
stop_words = get_stop_words('en')
nltk.download('punkt')
nltk.download('wordnet')
# nltk.download('all')

# This will prompt for authorization./
drive.mount('/content/drive')
# !ls "/content/drive/My Drive/Colab Notebooks"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading the dataset
acl_2017 = pd.read_csv("/content/drive/My Drive/Colab Notebooks (1)/PeerReadDataset/cleaned_data/clean_acl_2017.csv")
conll_2016 = pd.read_csv("/content/drive/My Drive/Colab Notebooks (1)/PeerReadDataset/cleaned_data/clean_conll_2016.csv")
iclr_2017 = pd.read_csv("/content/drive/My Drive/Colab Notebooks (1)/PeerReadDataset/cleaned_data/clean_iclr_2017.csv")

In [ ]:
# Dropping the index column
acl_2017.drop(acl_2017.columns[0],axis=1,inplace=True)
conll_2016.drop(conll_2016.columns[0],axis=1,inplace=True)
iclr_2017.drop(iclr_2017.columns[0],axis=1,inplace=True)

In [ ]:
# Concatenating all the dataframes
dataset = pd.concat([acl_2017, conll_2016, iclr_2017], axis=0, sort=True)
# dataset = acl_2017
dataset.reset_index(drop=True, inplace=True)
dataset.shape

(1638, 25)

In [ ]:
dataset.head(3)

,APPROPRIATENESS,Abstract,Accepted,CLARITY,Conclusion,Content,Discussion,IMPACT,Introduction,MEANINGFUL_COMPARISON,Name,ORIGINALITY,PRESENTATION_FORMAT,RECOMMENDATION,REPLICABILITY,REVIEWER_CONFIDENCE,Related_work,Result,SOUNDNESS_CORRECTNESS,SUBSTANCE,Title,Year,comments,is_meta_review,terms
0,5,We propose two novel methodologies for the aut...,1.0,4,"Our contributions are twofold. First, we devel...",Our first model is a pure neural language mode...,NaN,3.0,Poetry is an advanced form of linguistic commu...,5.0,660,4,Oral Presentation,4,NaN,3,Automatic poetry generation is an important ta...,In order to examine how effective our methodol...,3,5,Automatically Generating Rhythmic Verse with N...,2017.0,The paper presents two approaches for generati...,NaN,paper present two approach generating english ...
1,5,We propose two novel methodologies for the aut...,1.0,3,"Our contributions are twofold. First, we devel...",Our first model is a pure neural language mode...,NaN,3.0,Poetry is an advanced form of linguistic commu...,5.0,660,4,Poster,3,NaN,4,Automatic poetry generation is an important ta...,In order to examine how effective our methodol...,3,4,Automatically Generating Rhythmic Verse with N...,2017.0,The paper describes two methodologies for the ...,NaN,paper describes two methodology automatic gene...
2,5,"We investigate object naming, which is an impo...",1.0,3,NaN,NaN,NaN,3.0,NaN,3.0,489,5,Poster,4,NaN,4,NaN,NaN,5,5,Obtaining referential word meanings from visua...,NaN,COMMENTS AFTER AUTHOR RESPONSE:\n\nThanks for ...,NaN,comment author response thanks response partic...


In [ ]:
dataset = dataset.dropna(subset=['comments'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1635 entries, 0 to 1637
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   APPROPRIATENESS        1635 non-null   int64  
 1   Abstract               1633 non-null   object 
 2   Accepted               1596 non-null   float64
 3   CLARITY                1635 non-null   int64  
 4   Conclusion             1198 non-null   object 
 5   Content                1480 non-null   object 
 6   Discussion             371 non-null    object 
 7   IMPACT                 1635 non-null   float64
 8   Introduction           1429 non-null   object 
 9   MEANINGFUL_COMPARISON  1635 non-null   float64
 10  Name                   1635 non-null   int64  
 11  ORIGINALITY            1635 non-null   int64  
 12  PRESENTATION_FORMAT    1635 non-null   object 
 13  RECOMMENDATION         1635 non-null   int64  
 14  REPLICABILITY          1360 non-null   float64
 15  REVI

In [ ]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def remove_stopwords(s):
    """For removing stop words
    """
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s

def preprocess(text):
    
    # remove html tags
    text = strip_tags(text)
    
    # Convert to lowercase
    text = text.lower()
    
    # remove stopwords
    text = remove_stopwords(text)

    # Remove numbers from text 
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)

    # Remove punctuation & single letter words
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace(r'\b\w\b', '')
    text = text.replace(r'\s+', ' ')
    text = text.replace(r'\n', ' ')
    text = text.replace(r'\r\n', ' ')
    
    # Remove nan
    text = re.sub(r'\bnan\b', '', text)
    
    # Tokenize: Split the sentence into words & Lemmatize list of words and join
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])

    return text

In [ ]:
def get_required_data(content, text_fields):        
        content['terms'] = content[text_fields].fillna('').apply(' '.join, axis = 1)
        # cleaned_products = products.filter(['id', 'terms'])        
        return content

text_fields = ['comments']    
# text_fields = ['Abstract', 'Title', 'comments', 'Content', 'Result', 'Introduction', 'Conclusion', 'Related_work', 'Discussion']
cleaned_content = get_required_data(dataset, text_fields)

In [ ]:
cleaned_content['terms'] = cleaned_content['terms'].apply(preprocess)

In [ ]:
cleaned_content.head(2)

,APPROPRIATENESS,Abstract,Accepted,CLARITY,Conclusion,Content,Discussion,IMPACT,Introduction,MEANINGFUL_COMPARISON,Name,ORIGINALITY,PRESENTATION_FORMAT,RECOMMENDATION,REPLICABILITY,REVIEWER_CONFIDENCE,Related_work,Result,SOUNDNESS_CORRECTNESS,SUBSTANCE,Title,Year,comments,is_meta_review,terms
0,5,We propose two novel methodologies for the aut...,1.0,4,"Our contributions are twofold. First, we devel...",Our first model is a pure neural language mode...,NaN,3.0,Poetry is an advanced form of linguistic commu...,5.0,660,4,Oral Presentation,4,NaN,3,Automatic poetry generation is an important ta...,In order to examine how effective our methodol...,3,5,Automatically Generating Rhythmic Verse with N...,2017.0,The paper presents two approaches for generati...,NaN,the paper present two approach for generating ...
1,5,We propose two novel methodologies for the aut...,1.0,3,"Our contributions are twofold. First, we devel...",Our first model is a pure neural language mode...,NaN,3.0,Poetry is an advanced form of linguistic commu...,5.0,660,4,Poster,3,NaN,4,Automatic poetry generation is an important ta...,In order to examine how effective our methodol...,3,4,Automatically Generating Rhythmic Verse with N...,2017.0,The paper describes two methodologies for the ...,NaN,the paper describes two methodology for the au...


In [ ]:
cleaned_content.to_csv("/content/drive/My Drive/Colab Notebooks (1)/PeerReadDataset/cleaned_data/preprocessed_final_data.csv", index=False)